[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/juanchess98/Notebooks-Deep-Learning/blob/transfer_learning/TL_CASO_1.ipynb)

# Transfer Learning (Enfoque por cuadrantes o escenarios usando Pytorch)

Aprender una tarea nueva puede ser un proceso que lleve una considerable cantidad de tiempo. Sin embargo, los seres humanos tienen la capacidad de asociar nuevas tareas con otras que ya sabe como ejecutar y aplicar este conocimiento para reducir el tiempo y costo de aprendizaje. Por ejemplo, si una persona sabe como manejar una bicicleta, para esa persona será mucho más fácil aprender a manejar una motocicleta. Asimismo, una persona que domine el idioma español tendrá mucha más facilidad para aprender otro idioma similar, como el portugués, que una persona de origen alemán o japonés.

Este mismo concepto se puede aplicar a las máquinas inteligentes y se le conoce como *Transfer Learning* o Aprendizaje por transferencia. En este caso, un modelo desarrollado para una resolver una tarea puede usarse como punto de partida para resolver otra tarea similar.

Pueden presentarse diferentes escenarios en la aplicación de *transferencia de aprendizaje* a un dataset nuevo, pero generalmente se puede reducir a cuatro casos:

1. La nueva base de datos es numerosa y se parece muy poco a la base de datos del modelo pre-entrenado
2. La nueva base de datos es numerosa y parecida a la base de datos del modelo pre-entrenado
3. La nueva base de datos es pequeña y diferente de la base de datos del modelo pre-entrenado
4. La nueva base de datos es pequeña y parecida a la base de datos del modelo pre-entrenado

En la siguiente gráfica tipo plano cartesiano pueden visualizarce mejor los posibles escenarios, dividiendo el plano en cuadrantes que representan cada escenario y dónde en la dirección de crecimiento del eje $x$ se indica la similaridad de los dos datasets y el eje $y$ el tamaño del nuevo dataset.

**(Insertar imagen de cuadrantes aquí)**

Para cada escenario existe una metodología diferente a implementar y en el curso de este notebook trataremos de ejemplificar cada una con la ayuda de los frameworks de Keras y Pytorch.

## Caso 1. La nueva base de datos es numerosa y diferente a la del modelo pre-entrenado

Si la nueva base de datos es grande y no se parece a la original, se recomienda seguir los siguientes pasos:

1. Quitar la última capa totalmente conectada y añadir una nueva capa totalmente conectada cuya dimensión de salida se igual al número de clases de la nueva base de datos.

2. Aleatorizar los pesos de la nueva capa totalmente conectada e inicializarlos con valores aleatorios

3. Entrenar la red para actualizar los pesos de la nueva capa totalmente conectada.


In [ ]:
import time
import copy
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
import torchvision.transforms as transforms

### Cargar la base de datos DIFERENTE

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    model.to(device)

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
                dataloaders = trainloader
                '''for inputs, labels in trainloader:
                    inputs = inputs.to(device)
                    labels = labels.to(device)'''
            else:
                model.eval()   # Set model to evaluate mode
                dataloaders = testloader
                '''for inputs, labels in testloader:
                    inputs = inputs.to(device)
                    labels = labels.to(device)'''

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):

                    outputs = model(inputs).to(device)

                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        #loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()

            '''epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())'''

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

### Cargar el modelo preentrenado

In [ ]:
from torchvision import datasets, models, transforms

In [ ]:
pretrained_model = models.vgg16(pretrained=False)